# 第八章 类与对象

## 10.1 8.1 改变对象的字符串显示

要改变一个实例的字符串表示，可重新定义它的str () 和repr () 方法

In [1]:
class Pair:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __repr__(self):
        return 'Pair({0.x!r}, {0.y!r})'.format(self)
    def __str__(self):
        return '({0.x!s}, {0.y!s})'.format(self)

\_\_repr\_\_() 生成的文本字符串标准做法是需要让eval(repr(x)) == x 为真  
如果\_\_str\_\_() 没有被定义，那么就会使用\_\_repr\_\_() 来代替输出

## 10.2 8.2 自定义字符串的格式化

为了自定义字符串的格式化，我们需要在类上面定义\_\_format\_\_) 方法

In [2]:
_formats = {
'ymd' : '{d.year}-{d.month}-{d.day}',
'mdy' : '{d.month}/{d.day}/{d.year}',
'dmy' : '{d.day}/{d.month}/{d.year}'
}
class Date:
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day
    def __format__(self, code):
        if code == '':
            code = 'ymd'
        fmt = _formats[code]
        return fmt.format(d=self)

In [3]:
d = Date(2012, 12, 21)
format(d)

'2012-12-21'

In [4]:
format(d, 'mdy')

'12/21/2012'

In [5]:
'The date is {:mdy}'.format(d)

'The date is 12/21/2012'

参考来自的datetime模块中的代码

In [6]:
from datetime import date
d = date(2012, 12, 21)
format(d)

'2012-12-21'

In [7]:
format(d,'%A, %B %d, %Y')

'Friday, December 21, 2012'

In [9]:
'The end is {:%d %b %Y}. Goodbye'.format(d)

'The end is 21 Dec 2012. Goodbye'

## 10.3 8.3 让对象支持上下文管理协议

为了让一个对象兼容with 语句，你需要实现\_\_enter\_\_() 和\_\_exit\_\_() 方法

In [13]:
from socket import socket, AF_INET, SOCK_STREAM
class LazyConnection:
    def __init__(self, address, family=AF_INET, type=SOCK_STREAM):
        self.address = address
        self.family = family
        self.type = type
        self.sock = None
    def __enter__(self):
        if self.sock is not None:
            raise RuntimeError('Already connected')
        self.sock = socket(self.family, self.type)
        self.sock.connect(self.address)
        return self.sock
    def __exit__(self, exc_ty, exc_val, tb):
        self.sock.close()
        self.sock = None

In [14]:
from functools import partial
conn = LazyConnection(('www.python.org', 80))
# Connection closed
with conn as s:
    # conn.__enter__() executes: connection open
    s.send(b'GET /index.html HTTP/1.0\r\n')
    s.send(b'Host: www.python.org\r\n')
    s.send(b'\r\n')
    resp = b''.join(iter(partial(s.recv, 8192), b''))
    # conn.__exit__() executes: connection closed

多个with嵌套用，则需要作如下修改

In [17]:
from socket import socket, AF_INET, SOCK_STREAM
class LazyConnection:
    def __init__(self, address, family=AF_INET, type=SOCK_STREAM):
        self.address = address
        self.family = family
        self.type = type
        self.connections = []
    def __enter__(self):
        sock = socket(self.family, self.type)
        sock.connect(self.address)
        self.connections.append(sock)
        return sock
    def __exit__(self, exc_ty, exc_val, tb):
        self.connections.pop().close()

In [18]:
from functools import partial
conn = LazyConnection(('www.python.org', 80))
with conn as s1:
    pass
    with conn as s2:
        pass
    # s1 and s2 are independent sockets

## 10.4 8.4 创建大量对象时节省内存方法

对于主要是用来当成简单的数据结构的类而言，你可以通过给类添加slots 属
性来极大的减少实例所占的内存

In [19]:
class Date:
    __slots__ = ['year', 'month', 'day']
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

当你定义slots 后，Python 就会为实例使用一种更加紧凑的内部表示。实例通
过一个很小的固定大小的数组来构建，而不是为每个实例定义一个字典，这跟元组或
列表很类似。在slots 中列出的属性名在内部被映射到这个数组的指定小标上。使
用slots 一个不好的地方就是我们不能再给实例添加新的属性了，只能使用在slots
中定义的那些属性名。

## 10.5 8.5 在类中封装属性名

第一个约定是任何以单下划线开头的名字都应该是内部实现。

In [20]:
class A:
    def __init__(self):
        self._internal = 0 # An internal attribute
        self.public = 1 # A public attribute
    def public_method(self):
        '''
        A public method
        '''
        pass
    def _internal_method(self):
        pass

双下划线

In [21]:
class B:
    def __init__(self):
        self.__private = 0
    def __private_method(self):
        pass
    def public_method(self):
        pass
        self.__private_method()

使用双下划线开始会导致访问名称变成其他形式。比如，在前面的类B 中，私有
属性会被分别重命名为\_B\_\_private 和\_B\_\_private_method 。  
为什么要重命名，答案就是继承--这种属性通过继承时无法覆盖的

In [23]:
class C(B):
    def __init__(self):
        super().__init__()
        self.__private = 1 # Does not override B.__private
    # Does not override B.__private_method()
    def __private_method(self):
        pass

C 则会拥有父类的\_B\_\_private和自己的\_C\_\_private

In [ ]:
单划线和双划线: 大多数而言，你应该让你的非公共名称以单下划线开
头。但是，如果你清楚你的代码会涉及到子类，并且有些内部属性应该在子类中隐藏
起来，那么才考虑使用双下划线方案。???

## 10.6 8.6 创建可管理的属性

In [42]:
class Person:
    def __init__(self, first_name):
        self.first_name = first_name
        
    # Getter function
    @property
    def first_name(self):
        return self._first_name
    
    # Setter function
    @first_name.setter
    def first_name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._first_name = value
    # Deleter function (optional)
    @first_name.deleter
    def first_name(self):
        raise AttributeError("Can't delete attribute")

In [43]:
a = Person('Guido')
a.first_name # Calls the getter

'Guido'

\_first\_name 是实际存数据的地方

一个property 属性其实就是一系列相关绑定方法的集合。如果你去查看拥有
property 的类，就会发现property 本身的fget、fset 和fdel 属性就是类里面的普通方
法.这些函数在你访问property的时候回自动触发。只有当你确实需要对attribute 执行其他额外的操作的时候才应该使用到property。Properties 还是一种定义动态计算attribute 的方法。这种类型的attributes 并不会被实际的存储，而是在需要的时候计算出来。

In [46]:
Person.first_name.fget
Person.first_name.fset
Person.first_name.fdel

<function __main__.Person.first_name(self)>

In [48]:
import math
class Circle:
    def __init__(self, radius):
        self.radius = radius
    @property
    def area(self):
        return math.pi * self.radius ** 2
    @property
    def diameter(self):
        return self.radius ** 2
    @property
    def perimeter(self):
        return 2 * math.pi * self.radius

In [49]:
A = Circle(5)

In [51]:
A.area

78.53981633974483

## 10.7 8.7 调用父类方法

为了调用父类(超类) 的一个方法，可以使用super() 函数

In [54]:
class A:
    def spam(self):
        print('A.spam')
class B(A):
    def spam(self):
        print('B.spam')
        super().spam() # Call parent spam()

In [ ]:
super() 函数的一个常见用法是在init () 方法中确保父类被正确的初始化了

In [ ]:
super() 的另外一个常见用法出现在覆盖Python 特殊方法的代码中

## 10.8 8.8 子类中扩展property

In [55]:
class Person:
    def __init__(self, name):
        self.name = name
    # Getter function
    @property
    def name(self):
        return self._name
    # Setter function
    @name.setter
    def name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._name = value
    # Deleter function
    @name.deleter
    def name(self):
        raise AttributeError("Can't delete attribute")